In [231]:
# !pip install mlforecast lightgbm -q

In [232]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme()

from lightgbm import LGBMRegressor

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from scipy.stats import uniform, loguniform, randint

import optuna

from mlforecast import MLForecast
from mlforecast.lag_transforms import (
    ExpandingMean,
    RollingMean,
    ExponentiallyWeightedMean,
)
from mlforecast.target_transforms import Differences
from mlforecast.auto import (
    AutoLightGBM,
    AutoMLForecast,
    AutoModel,
    AutoRidge,
    ridge_space,
)

from functools import partial
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import mape, mase, mae, rmse

import time
import datetime

In [233]:
forecast_horizon = 24

# Import Data

In [234]:
df = pd.read_csv(
    "../data/consumption.csv", usecols=["prediction_unit_id", "datetime", "target"]
)[["prediction_unit_id", "datetime", "target"]].rename(
    columns={"prediction_unit_id": "unique_id", "datetime": "ds", "target": "y"}
)
df["ds"] = pd.to_datetime(df["ds"])
df.head()

,unique_id,ds,y
0,0,2021-09-01,96.590
1,1,2021-09-01,17.314
2,2,2021-09-01,656.859
3,3,2021-09-01,59.000
4,4,2021-09-01,501.760


In [235]:
df.shape

(1009176, 3)

In [236]:
df["y"] = df["y"].ffill()
df.isna().sum()

unique_id    0
ds           0
y            0
dtype: int64

# Train/Test split

In [237]:
# taking the last 60 days for test
test_duration = 24 * 60
for i in df["unique_id"].unique():
    if i == df["unique_id"].unique()[0]:
        df_test = df[df["unique_id"]==i][-test_duration:]
        continue
    df_test = pd.concat([df_test, df[df["unique_id"]==i][-test_duration:]])
    
df_test.sort_index(inplace=True)
print(df_test.shape)
df_test.head()

(99360, 3)


,unique_id,ds,y
684254,68,2022-11-10 00:00:00,28.124
684323,68,2022-11-10 01:00:00,28.020
684392,68,2022-11-10 02:00:00,28.741
684461,68,2022-11-10 03:00:00,31.947
684530,68,2022-11-10 04:00:00,36.197


In [238]:
train_idx = [idx for idx in df.index if idx not in df_test.index]
df_train = df.loc[train_idx]
df_train.shape
df_train.head()

,unique_id,ds,y
0,0,2021-09-01,96.590
1,1,2021-09-01,17.314
2,2,2021-09-01,656.859
3,3,2021-09-01,59.000
4,4,2021-09-01,501.760


In [239]:
df.shape[0] == df_train.shape[0] + df_test.shape[0]

True

In [240]:
df.shape[1] == df_train.shape[1] == df_test.shape[1]

True

In [241]:
test_size = df_test.shape[0] / (df_train.shape[0] + df_test.shape[0])
print(f"test size : {round(test_size, 3)*100}0%")

test size : 9.80%


# Preprocessing

In [242]:
fcst = MLForecast(
    models=[],
    freq="h",
    # target_transforms=[Differences([24])],
    lags=[i + 1 for i in range(47)],
    lag_transforms={
        1: [ExpandingMean()],
        1: [RollingMean(window_size=24)],
        24: [RollingMean(window_size=24)],
        # 24: [RollingMean(window_size=48)],
    },
    date_features=["month", "dayofweek", "hour"],
)

X_train = fcst.preprocess(df_train).rename(columns={"y": "lag0"})
X_train, y_train = X_train.align(
    df_train.groupby("unique_id")["y"].shift(-24).rename("lead24").dropna(),
    axis=0,
    join="inner",
)
print(X_train.shape, y_train.shape)
X_train.head(3)

(904917, 55) (904917,)


,unique_id,ds,lag0,lag1,lag2,lag3,lag4,lag5,lag6,lag7,...,lag43,lag44,lag45,lag46,lag47,rolling_mean_lag1_window_size24,rolling_mean_lag24_window_size24,month,dayofweek,hour
2867,0,2021-09-02 23:00:00,120.540,134.986,150.412,152.763,136.130,121.033,80.621,43.428,...,88.184,87.955,91.594,77.691,96.590,87.588333,79.969750,9,3,23
2868,1,2021-09-02 23:00:00,19.430,21.577,24.309,27.201,25.419,18.060,16.228,10.614,...,18.225,14.271,16.510,15.872,17.314,15.725333,15.106667,9,3,23
2869,2,2021-09-02 23:00:00,748.504,920.535,1000.499,988.047,877.168,656.989,489.864,278.374,...,607.308,622.824,598.450,595.498,656.859,556.726667,497.720500,9,3,23


In [243]:
y_train.head(3)

2867    139.929
2868     27.217
2869    898.365
Name: lead24, dtype: float64

In [244]:
X_test = fcst.preprocess(df_test).rename(columns={"y": "lag0"})
X_test, y_test = X_test.align(
    df_test.groupby("unique_id").shift(-24).dropna(),
    axis=0,
    join="inner",
)
y_test["unique_id"] = X_test["unique_id"]
print(X_test.shape, y_test.shape)

(94461, 55) (94461, 3)


# HPO

## Default optimization

In [28]:
optuna.logging.set_verbosity(optuna.logging.ERROR)
auto_mlf = AutoMLForecast(
    models={
        "lgb": AutoLightGBM(),
        # "ridge": AutoRidge(),
    },
    freq="h",
    season_length=24,
)

In [29]:
%%time
auto_mlf.fit(
    df_train,
    n_windows=10,
    h=forecast_horizon,
    num_samples=2,  # number of trials to run
    # **col_params
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/utilsforecast/processing.py:737: UserWarning: The following series are too short for the window and will be dropped: [68]
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/utilsforecast/processing.py:737: UserWarning: The following series are too short for the window and will be dropped: [68]
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/utilsforecast/processing.py:737: UserWarning: The following series are too short for the window and will be dropped: [68]
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/utilsforecast/processing.py:737: UserWarning: The following series are too short for the window and will be dropped: [68]
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/mlforecast/core.py:579: UserWarning: Found null values in lag24.
  warnings.warn(f'Found null values in {", "

CPU times: user 1min 54s, sys: 1.49 s, total: 1min 56s
Wall time: 14.8 s


AutoMLForecast(models={'lgb': AutoModel(model=LGBMRegressor)})

### Evaluation

https://nixtlaverse.nixtla.io/mlforecast/docs/how-to-guides/cross_validation.html

## Tuning lgbm parameters

In [51]:
def my_lgb_config(trial: optuna.Trial):
    return {
        "learning_rate": 0.05,
        "verbosity": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 512, log=True),
        "objective": trial.suggest_categorical("objective", ["l1", "l2", "mape"]),
    }


my_lgb = AutoModel(
    model=LGBMRegressor(),
    config=my_lgb_config,
)

In [27]:
%%time
auto_mlf = AutoMLForecast(
    models={"my_lgb": my_lgb},
    freq="h",
    season_length=24,
).fit(
    df_train,
    n_windows=2,
    h=forecast_horizon,
    num_samples=2,
)

NameError: name 'my_lgb' is not defined

### Evaluation

https://nixtlaverse.nixtla.io/mlforecast/docs/how-to-guides/cross_validation.html

## Tuning ridge parameters

In [34]:
ridge_pipeline = make_pipeline(
    ColumnTransformer(
        [("encoder", OneHotEncoder(), ["unique_id"])],
        remainder="passthrough",
    ),
    Ridge(),
)
my_auto_ridge = AutoModel(
    ridge_pipeline,
    # the space must have the name of the estimator followed by the parameter
    # you could also tune the encoder here
    lambda trial: {f"ridge__{k}": v for k, v in ridge_space(trial).items()},
)

In [35]:
%%time
auto_mlf = AutoMLForecast(
    models={"ridge": my_auto_ridge},
    freq="h",
    season_length=24,
    fit_config=lambda trial: {"static_features": ["unique_id"]},
).fit(
    df_train,
    n_windows=2,
    h=forecast_horizon,
    num_samples=2,
)

CPU times: total: 8.27 s
Wall time: 8.75 s


### Evaluation

https://nixtlaverse.nixtla.io/mlforecast/docs/how-to-guides/cross_validation.html

## Tuning features

In [40]:
def my_init_config(trial: optuna.Trial):
    lag_transforms = [
        ExponentiallyWeightedMean(alpha=0.3),
        RollingMean(window_size=24 * 7, min_samples=1),
    ]
    lag_to_transform = trial.suggest_categorical("lag_to_transform", [24, 48])
    return {
        "lags": [24 * i for i in range(1, 7)],  # this won't be tuned
        "lag_transforms": {lag_to_transform: lag_transforms},
    }

In [41]:
%%time
auto_mlf = AutoMLForecast(
    # models={'ridge': my_auto_ridge},
    # fit_config=lambda trial: {'static_features': ['unique_id']}
    models=[AutoRidge()],
    freq="h",
    season_length=24,
    init_config=my_init_config,
).fit(
    df_train,
    n_windows=2,
    h=forecast_horizon,
    num_samples=2,
)

CPU times: total: 3.75 s
Wall time: 3.97 s


In [42]:
preds = auto_mlf.predict(forecast_horizon)
preds

,unique_id,ds,AutoRidge
0,0,2023-05-30 00:00:00,431.601917
1,0,2023-05-30 01:00:00,381.425067
2,0,2023-05-30 02:00:00,376.351815
3,0,2023-05-30 03:00:00,382.440528
4,0,2023-05-30 04:00:00,359.983041
...,...,...,...
3307,68,2023-05-31 19:00:00,1.329570
3308,68,2023-05-31 20:00:00,2.455559
3309,68,2023-05-31 21:00:00,6.480433
3310,68,2023-05-31 22:00:00,9.512457


### Evaluation

https://nixtlaverse.nixtla.io/mlforecast/docs/how-to-guides/cross_validation.html